In [1]:
from google.colab import files
files.upload()

Saving en_data.csv to en_data.csv
Saving Friends.zip to Friends.zip


{'Friends.zip': b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1bw\x06M\x9c\xd9\x99\xc7hu\x00\x00\x10\x80\x03\x00\x10\x00\x00\x00friends_dev.json\xec\xbd[w\x1bG\xd6%\xf8>\xbf"\xe8\x17\xce\x03\xc8\x11({}\xd5=\x0fZ\x92\xec\xb2\xe8\xb2%\xb7%\x7f\x1a\xaf\xf6<\x04\x80\x002\xcd\xcc\x0cT^\x08\xa1z\xcd\x7f\x9f\xb3\xcf\x89L$HWf$\x10\xe9\xc4t\x8f\xab,Q$E\xc7\xf5\xc4\xb9\xec\xb3\xf7\x7f\xff\xdf\x14\xfd\xf3\xdf\xf9W\xfc\xf3?\x9a\x8f\xf0\xcfW\xc5\xd6\xe8\x07\x93\x7f\xf5_\xd5W?G\xd6,\xccW\xb3\xe3o\xa8\xca\xd2\xe4:[\x1a|\xcb\x87H\xa5{\xf5\xbd]\xcdTd~\xaf^\xbc\xf8/w\x85JlQ\xaa\xb8\xbcU\xef\x9aO\x95\xb6\xd4I\xb2o\xbe\xf4\xf4\x87\x9a\xd4\x96\xb1\xcd\xf0#3\x9b\xddd\xa6*s\x9d<\xfd.\x9de\xf4s\xeao|\xf1\xe2\xc5\xdd\xfc\xee\xc5W\xcd\xf7\xfc?3\x8fI\xfdd\xb3x\xa9;\'\xf59\xd2\xe5\xab\x8e\x11\x16U\xbe\xcd\xe3\xe2\xd9\xca\x1c\x0foN\xe3\x9b\xbf\x1c:\xbc_lQt\xafx~\xa5>\xe43\xb53ji\xabd\xa56\x96VW\x95\x91Q\x0b\x9d=\xcc\xd4\x92\xd6\xd8([\xe5J/\xe9;\xb2\xb2P:[\xa9eU\xe2\x9bRe\xd7k\xa5\xf9cU\xd0w-M\xe7f\xf8l\xc4K\xec\xc4\x8b\xa

In [2]:
!unzip Friends.zip

Archive:  Friends.zip
  inflating: friends_dev.json        
  inflating: friends_test.json       
  inflating: friends_train.json      


In [3]:
!pip install transformers

     |████████████████████████████████| 1.5MB 9.5MB/s 
     |████████████████████████████████| 2.9MB 23.5MB/s 
     |████████████████████████████████| 890kB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1b498e756bfaab4dac71a737f64eb48d0c4e0845d9e7f07123abe254055f4efc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import tensorflow as tf
import torch

from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import time
import datetime
import json


In [5]:
def jsonToDf(file_name):
  with open(file_name, encoding = 'utf-8', mode = 'r') as file:
    json_array = json.load(file)
  
  result = pd.DataFrame.from_dict(json_array[0])

  is_first = True
  for array in json_array:
    if is_first:
      is_first = False
      continue
    
    temp_df = pd.DataFrame.from_dict(array)
    result = result.append(temp_df, ignore_index = True)

  return result

In [6]:
train = jsonToDf('friends_train.json')
dev = jsonToDf('friends_dev.json')
train2 = jsonToDf('friends_test.json')

train = train.append(train2, ignore_index = True)

In [7]:
test = pd.read_csv('en_data.csv')

In [8]:
test.head()

id  ...                                          utterance
0   0  ...                      Alright, whadyou do with him?
1   1  ...                                  Oh! You're awake!
2   2  ...  Then you gotta come clean with Ma! This is not...
3   3  ...                                  Yeah, but this is
4   4  ...          I don't wanna hear it! Now go to my room!

[5 rows x 5 columns]

In [9]:
print(train.shape)
print(dev.shape)
print(test.shape)

(13325, 4)
(1178, 4)
(1623, 5)


In [62]:
MAX_LEN = 100

def getInputsAndLabels(dataset):
  data = dataset.copy(deep=True)
  data['utterance'] = data['utterance'].str.lower()

  utterances = data['utterance']
  utterances = ["[CLS] " + str(utterance) + " [SEP]" for utterance in utterances]
  
  encoder = LabelEncoder()
  labels = data['emotion'].values
  encoder.fit(labels)
  labels = encoder.transform(labels)
  #tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, labels, attention_masks
 

In [63]:
def getInputsFromTest(dataset):
  data = dataset.copy(deep=True)
  data['utterance'] = data['utterance'].str.lower()

  utterances = data['utterance']
  utterances = ["[CLS] " + str(utterance) + " [SEP]" for utterance in utterances]
  
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
  #tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks

In [64]:
def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.id.tolist()
  return torch.tensor(input_index)

In [65]:
train_inputs, train_labels, train_masks = getInputsAndLabels(train)
dev_inputs, dev_labels, dev_masks = getInputsAndLabels(dev)
test_inputs, test_masks = getInputsFromTest(test)

In [66]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

dev_inputs = torch.tensor(dev_inputs)
dev_labels = torch.tensor(dev_labels)
dev_masks = torch.tensor(dev_masks)

test_index = getIndex(test)
test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)

print(test_inputs[0])
print(test_index[0])
print(test_masks[0])

tensor([  101, 10164, 33661,   117,   191, 33796, 15594, 10138, 10149, 10169,
        10957,   136,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
tensor(0)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [67]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

test_data = TensorDataset(test_index, test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## **모델 구현**

BERT

In [ ]:
import tensorflow as tf

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [68]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [69]:
# 분류를 위한 BERT 모델 생성
#model = ElectraForSequenceClassification.from_pretrained('google/electra-small-generator', num_labels=8)
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=8)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [70]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8
                )

epochs = 5

total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

## **학습**

In [71]:
from sklearn.metrics import f1_score

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def getF1Score(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  return f1_score(labels_flat, pred_flat, average = None)

In [72]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [73]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()
    # 로스 초기화
    total_loss = 0
    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        # Forward 수행      
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        # 로스 구함
        loss = outputs[0]
        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()
        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy, eval_f1 = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in dev_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
     
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        # tmp_eval_f1 = getF1Score(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        # eval_f1 += tmp_eval_f1
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    # print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...

  Average training loss: 1.30
  Training epcoh took: 0:01:22

Running Validation...
  Accuracy: 0.54
  Validation took: 0:00:02

======== Epoch 2 / 5 ========
Training...

  Average training loss: 1.12
  Training epcoh took: 0:01:23

Running Validation...
  Accuracy: 0.56
  Validation took: 0:00:02

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.99
  Training epcoh took: 0:01:22

Running Validation...
  Accuracy: 0.56
  Validation took: 0:00:02

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.88
  Training epcoh took: 0:01:23

Running Validation...
  Accuracy: 0.56
  Validation took: 0:00:02

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.79
  Training epcoh took: 0:01:23

Running Validation...
  Accuracy: 0.56
  Validation took: 0:00:02

Training complete!


In [74]:
tmp_test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)
test_result = test.copy(deep = True)
test_result = test_result.drop(columns = ['i_dialog', 'i_utterance', 'speaker'])
test_result['Expected'] = 'default'

encoder = LabelEncoder()
labels = train['emotion'].values
encoder.fit(labels)
labels = encoder.transform(labels)


for step, batch in enumerate(tmp_test_dataloader):
    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_index, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Expected'][idx] = encoder.classes_[np.argmax(logits)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [75]:
test_result.tail()

id    utterance     Expected
1618  1618        Nooo.  non-neutral
1619  1619    Hi, Kate!          joy
1620  1620  Hi, Lauren.      neutral
1621  1621  Hi, Lauren.      neutral
1622  1622     Hi, pig!          joy

In [76]:
test_result = test_result.drop(columns = ['utterance'])

In [78]:
test_csv = test_result.to_csv('subfriends21.csv')

In [79]:
from google.colab import files

files.download('subfriends21.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>